In [46]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets import Checkbox, VBox,HBox,Label, interact,Box



In [19]:
# import DiCE
import dice_ml
from dice_ml.utils import helpers # helper functions

# Tensorflow libraries
import tensorflow as tf
from tensorflow import keras

dataset = helpers.load_adult_income_dataset()

In [90]:
feature_names=['age', 'workclass', 'education', 'marital_status', 'occupation', 'race', 'gender', 'hours_per_week']
cont_feat=['age','hours_per_week']

In [157]:
from ipywidgets import interact_manual
from IPython.display import display
#from ipywidgets import TwoByTwoLayout
from ipywidgets import HBox, Label

print('1. Select Features to use:                      2. Feature Weights:                    3. Query:')

feature_children=[]
for i in feature_names:
    new_check1=Checkbox(description=i, value=True)
    feature_children.append(new_check1)


feature_vb = VBox(children = feature_children)


#display(feature_vb)


def f(a,b,c,d,e,f,g,h):
    use=[a,b,c,d,e,f,g,h]
    
    using=[]
    for i in feature_names:
        using.append(i)
    #print(use)
    for i in range(len(use)):
        #print(use[i])
        if use[i] is False:
            using.remove(feature_names[i])
            
    #print(using)
    
    feat_child=[]
    for i in using:
        new_slider=widgets.FloatSlider(description=i, value=1, min=0.1,max=5)
        feat_child.append(new_slider)
        
    vb = VBox(children = feat_child)
    
    
    
    inputq=[]
    #print('Input Query:')

    index=-1
    for i in using:
        index+=1
        if i in cont_feat:
            cont_text=widgets.Text(value='', description=i)
            #display(cont_text)
            inputq.append(cont_text)
        
        else:
            noncont_dropdown=widgets.Dropdown(description=i,options=dataset[i].unique())
            #display(noncont_dropdown)
            inputq.append(noncont_dropdown)
            
    new_children1=[]
    for i in inputq:
        new_children1.append(i)
        
    new_children=VBox(children=new_children1)
        
    display(HBox([feature_vb,vb,new_children]))
            

out=widgets.interactive_output(f,{'a':feature_children[0],'b':feature_children[1],'c':feature_children[2],'d':feature_children[3],'e':feature_children[4],'f':feature_children[5],'g':feature_children[6],'h':feature_children[7]})
display(out)






# PROX/DIV WEIGHTS
prox = widgets.FloatSlider(description='Proximity Weight', max=10, value=0.5)#default for prox in DiCE is 0.5
div = widgets.FloatSlider(description='Diversity Weight', max=10, value=1)#default for div in DiCE is 1

vb_proxdiv = VBox(children = [prox, div])
tune_proxdiv = Checkbox(description='4. Tune proximity/diversity?', indent=False)

def add_2(button):
    if button['new']:
        vb_proxdiv.children = [prox, div]
        display(vb_proxdiv)
    else:
        vb_proxdiv.children=[]
        #TwoByTwoLayout(top_left=tune_proxdiv,
         #      top_right=num_exp,
          #     bottom_right=highlight_changes)

tune_proxdiv.observe(add_2, names='value')

#NUMBER OF EXPLANATIONS
num_exp = widgets.IntSlider(description='5. Num of Explanations', min=1, max=5,style = {'description_width': 'initial'}, indent=False)
  
#HIGHLIGHT CHANGEs
highlight_changes=widgets.Checkbox(
    #value=False,
    description='6. Only highlight changes in explanations?',
    disabled=False,
    indent=False)

       
#DISPLAYING LAST 3 WIDGETS
display(HBox([tune_proxdiv, num_exp, highlight_changes]))


1. Select Features to use:                      2. Feature Weights:                    3. Query:


Output()

In [154]:
feat_child=[Label('Feauture Weights:')]
for i in feature_names:
    new_slider=widgets.FloatSlider(description=i, value=1, max=5)
    feat_child.append(new_slider)
    #display(new_slider)
        
weights=VBox(children=feat_child)
        
        
        
inputq=[Label('Input Query:')]
#print('Input Query:')

index=-1
for i in feature_names:
    index+=1
    if i in cont_feat:
        cont_text=widgets.Text(value='', description=i)
        #display(cont_text)
        inputq.append(cont_text)

    else:
        noncont_dropdown=widgets.Dropdown(description=i,options=dataset[i].unique())
        #display(noncont_dropdown)
        inputq.append(noncont_dropdown)
        
query=VBox(children=inputq)

     
display(HBox(children=[weights,query]))






# PROX/DIV WEIGHTS
prox = widgets.FloatSlider(description='Proximity Weight', max=10, value=0.5)#default for prox in DiCE is 0.5
div = widgets.FloatSlider(description='Diversity Weight', max=10, value=1)#default for div in DiCE is 1

vb_proxdiv = VBox(children = [prox, div])
tune_proxdiv = Checkbox(description='4. Tune proximity/diversity?', indent=False)

def add_2(button):
    if button['new']:
        vb_proxdiv.children = [prox, div]
        display(vb_proxdiv)
    else:
        vb_proxdiv.children=[]
        #TwoByTwoLayout(top_left=tune_proxdiv,
         #      top_right=num_exp,
          #     bottom_right=highlight_changes)

tune_proxdiv.observe(add_2, names='value')

#NUMBER OF EXPLANATIONS
num_exp = widgets.IntSlider(description='5. Num of Explanations', min=1, max=5,style = {'description_width': 'initial'}, indent=False)
  
#HIGHLIGHT CHANGEs
highlight_changes=widgets.Checkbox(
    #value=False,
    description='6. Only highlight changes in explanations?',
    disabled=False,
    indent=False)

       
#DISPLAYING LAST 3 WIDGETS
display(HBox([tune_proxdiv, num_exp, highlight_changes]))


In [127]:
v=widgets.RadioButtons(description="Query Input:",
    options=['From Dataset', 'Manually Enter'],layout={'width': 'max-content'}
        )

display(v)

RadioButtons(description='Query Input:', layout=Layout(width='max-content'), options=('From Dataset', 'Manuall…